In [1]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset
import numpy as np
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

/home/jaeyeob/anaconda3/envs/lstm/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

# MNIST 데이터 불러오기
(X_train_full, y_train_full), (X_test_full, y_test_full) = mnist.load_data()

# 데이터 정규화 (0-255 값을 0-1 사이로)
X_train_full = X_train_full.astype('float32') / 255.0
X_test_full = X_test_full.astype('float32') / 255.0

#torch.transform / normalize // imagenet mean sd 

# 숫자 1과 7만 선택하는 마스크 생성
train_mask = np.isin(y_train_full, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
X_train, y_train = X_train_full[train_mask], y_train_full[train_mask]

test_mask = np.isin(y_test_full, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
X_test, y_test = X_test_full[test_mask], y_test_full[test_mask]

# 시드 고정 (예: 42로 고정)
np.random.seed(42)

# 2000개의 데이터를 무작위로 선택
num_samples = 400
indices = np.random.choice(len(X_train), num_samples, replace=False)
X_sampled, y_sampled = X_train[indices], y_train[indices]

# 2000개의 샘플에서 train/test 데이터 분할 (80% train, 20% test 비율로 나눔)
X_train, X_test, y_train, y_test = train_test_split(
    X_sampled, y_sampled, stratify=y_sampled,test_size=0.2, random_state=42
)

# y_train과 y_test에서 7을 2로 변환
# y_train = np.where(y_train == 7, 2, y_train)
# y_test = np.where(y_test == 7, 2, y_test)

# 결과
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")


2024-11-28 16:17:25.038814: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-28 16:17:25.059468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 16:17:25.085317: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 16:17:25.101498: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 16:17:25.126553: I tensorflow/core/platform/cpu_feature_guar

Training set size: 320
Test set size: 80


In [3]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [4]:
X_train.shape

(320, 28, 28)

In [5]:
# 2x2 크기로 겹치지 않게 패치로 나누는 함수
def split_into_non_overlapping_patches(image, patch_size=(4, 4)):
    patches = []
    for i in range(0, image.shape[0], patch_size[0]):
        for j in range(0, image.shape[1], patch_size[1]):
            patch = image[i:i+patch_size[0], j:j+patch_size[1]].flatten()
            patches.append(patch)
    return np.array(patches)

In [6]:
# 각 이미지를 2x2 겹치지 않는 패치로 나누기
X_train = np.array([split_into_non_overlapping_patches(img) for img in X_train])
X_test = np.array([split_into_non_overlapping_patches(img) for img in X_test])

In [7]:
# y_train이 numpy 배열일 경우
unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

{0.0: 2880, 1.0: 320}


In [8]:
# y_train이 numpy 배열일 경우
unique, counts = np.unique(y_test, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

{0.0: 720, 1.0: 80}


In [9]:
X_train.shape

(320, 49, 16)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Self-Attention and Binary Classifier from previous code
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert self.head_dim * heads == embed_size, "Embedding size must be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask=None):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split embedding into multiple heads
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

# Classifier model
class SimpleSelfAttentionClassifier(nn.Module):
    def __init__(self, embed_size=16, heads=2, num_classes=10):
        super(SimpleSelfAttentionClassifier, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.fc = nn.Linear(embed_size * 49, num_classes)  # 49 is the sequence length in X_train

    def forward(self, x):
        attention_output = self.attention(x, x, x)
        attention_output = attention_output.flatten(start_dim=1)  # Flatten to feed into classifier
        out = self.fc(attention_output)
        return out

In [11]:
# Model, loss, optimizer
model = SimpleSelfAttentionClassifier(embed_size=16, heads=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
# Print out the parameters and their shapes
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter name: {name}")
        print(f"Shape: {param.shape}")
        print(f"Number of parameters: {param.numel()}")
        print("-" * 50)

# Total number of parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable parameters: {total_params}")


Parameter name: attention.values.weight
Shape: torch.Size([8, 8])
Number of parameters: 64
--------------------------------------------------
Parameter name: attention.keys.weight
Shape: torch.Size([8, 8])
Number of parameters: 64
--------------------------------------------------
Parameter name: attention.queries.weight
Shape: torch.Size([8, 8])
Number of parameters: 64
--------------------------------------------------
Parameter name: attention.fc_out.weight
Shape: torch.Size([16, 16])
Number of parameters: 256
--------------------------------------------------
Parameter name: attention.fc_out.bias
Shape: torch.Size([16])
Number of parameters: 16
--------------------------------------------------
Parameter name: fc.weight
Shape: torch.Size([10, 784])
Number of parameters: 7840
--------------------------------------------------
Parameter name: fc.bias
Shape: torch.Size([10])
Number of parameters: 10
--------------------------------------------------
Total number of trainable parameter

In [13]:
from torchsummary import summary
summary(model, input_size=(49, 16))  # (sequence_length, embed_size)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 49, 16]             272
     SelfAttention-2               [-1, 49, 16]               0
            Linear-3                   [-1, 10]           7,850
Total params: 8,122
Trainable params: 8,122
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.03
Estimated Total Size (MB): 0.05
----------------------------------------------------------------


/home/jaeyeob/anaconda3/envs/lstm/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [13]:
def multi_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    pred_soft = torch.log_softmax(preds, dim=1)
    _, pred_index = torch.max(pred_soft, dim=1)
    print(pred_index)
    y = [int(w.argmax()) for w in y]
    y = torch.tensor(y, dtype=float)
    print(y)
    correct_pred = (pred_index == y).float()
    #round predictions to the closest integer
    
    acc = correct_pred.sum()/len(correct_pred)

    return acc

In [14]:
# # Define your binary accuracy function
# def binary_accuracy(preds, y):
#     rounded_preds = torch.round(torch.sigmoid(preds))
#     correct = (rounded_preds == y).float()
#     acc = correct.sum() / len(correct)
    
#     return acc

In [15]:
for iepoch in tqdm(range(30)):
    optimizer.zero_grad()
    X_tensor=torch.tensor(X_train)
    predictions=model(X_tensor.float()).squeeze(1)
    #predictions=torch.sign(predictions)
    #print(predictions)
    label=torch.tensor(y_train)
    #for i in range(len(label)):
    #    if label[i]==1:
    #        label[i] = 0
    #    else:
    #        label[i]=1
    #print(label)
    loss = criterion(predictions, label.float())
    acc = multi_accuracy(predictions, label)
    print('')
    print('Accuracy:',acc)
    print('')
    print(loss)
    loss.backward()
    optimizer.step()

  0%|          | 0/30 [00:00<?, ?it/s]/home/jaeyeob/anaconda3/envs/lstm/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
 60%|██████    | 18/30 [00:00<00:00, 89.97it/s]

tensor([5, 7, 5, 4, 5, 5, 7, 2, 7, 5, 4, 4, 5, 4, 2, 0, 5, 5, 7, 5, 2, 7, 5, 5,
        5, 5, 2, 5, 5, 5, 0, 4, 5, 5, 7, 5, 5, 5, 7, 2, 4, 5, 5, 5, 5, 5, 5, 5,
        4, 4, 5, 2, 2, 4, 5, 5, 7, 4, 4, 2, 2, 5, 5, 5, 5, 2, 4, 2, 5, 5, 5, 5,
        5, 5, 2, 5, 5, 5, 2, 5, 5, 7, 5, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 4, 5, 5, 2, 5, 4, 2, 4, 5, 2, 5, 7, 5, 2, 5, 7, 5, 5, 7, 5, 5, 5, 7,
        2, 4, 2, 4, 5, 5, 5, 5, 5, 4, 2, 5, 4, 5, 5, 2, 5, 5, 5, 5, 7, 4, 4, 5,
        5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 2, 5, 2, 5, 7, 5, 7, 5, 7, 5, 5, 4, 5, 1,
        5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 2, 5, 4, 5, 7, 5, 5, 4, 4, 5, 5,
        4, 5, 4, 5, 5, 4, 4, 5, 4, 7, 2, 5, 5, 7, 4, 2, 2, 5, 5, 5, 2, 5, 5, 5,
        7, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 4, 5, 4, 4, 0,
        5, 7, 4, 5, 4, 7, 5, 1, 4, 5, 4, 5, 5, 5, 4, 5, 7, 5, 5, 7, 5, 5, 5, 5,
        7, 5, 7, 5, 5, 5, 2, 5, 2, 5, 5, 5, 2, 4, 5, 5, 5, 4, 4, 5, 5, 5, 4, 5,
        5, 7, 5, 4, 5, 5, 5, 5, 5, 2, 4,

100%|██████████| 30/30 [00:00<00:00, 91.69it/s]

tensor([4, 4, 0, 4, 0, 4, 4, 1, 4, 4, 4, 0, 0, 0, 1, 0, 4, 9, 0, 0, 1, 4, 9, 0,
        4, 4, 1, 1, 4, 0, 4, 0, 4, 4, 4, 4, 0, 4, 0, 1, 0, 9, 4, 0, 0, 0, 0, 9,
        4, 4, 4, 4, 4, 0, 4, 0, 9, 0, 0, 1, 1, 4, 0, 0, 4, 4, 4, 4, 4, 4, 4, 0,
        4, 1, 1, 0, 4, 0, 1, 0, 1, 9, 0, 0, 0, 0, 4, 4, 0, 9, 0, 0, 4, 4, 4, 0,
        0, 0, 5, 0, 1, 4, 0, 1, 8, 0, 1, 4, 0, 4, 1, 0, 4, 4, 4, 4, 0, 0, 4, 0,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 4, 0, 4, 0, 1, 0, 4, 4, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 4, 1, 0, 0, 4, 4, 1, 0, 1, 0, 0, 0, 4, 9, 4, 0, 0, 0, 4, 1,
        0, 0, 0, 0, 4, 4, 4, 0, 0, 0, 9, 0, 0, 1, 0, 0, 0, 4, 4, 1, 0, 0, 4, 0,
        0, 4, 0, 0, 4, 0, 0, 1, 0, 4, 1, 4, 0, 4, 0, 1, 1, 0, 1, 0, 1, 4, 0, 9,
        4, 4, 0, 4, 0, 9, 0, 0, 0, 4, 0, 4, 0, 4, 4, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        4, 4, 4, 4, 0, 4, 0, 1, 0, 1, 0, 1, 0, 4, 0, 0, 4, 1, 4, 0, 1, 4, 9, 4,
        4, 0, 4, 4, 0, 1, 1, 9, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 9, 4, 4, 0, 0,
        4, 4, 0, 4, 0, 4, 4, 4, 0, 1, 0,

In [16]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, average_precision_score

# Convert test data to tensor
X_tensor = torch.tensor(X_test, dtype=torch.float32)
predictions = model(X_tensor)

# Convert y_train to tensor and change labels from 1 to 0, and 7 to 1
label=torch.tensor(y_test)
# label = torch.tensor(y_test, dtype=torch.long).clone()  # Ensure labels are LongTensor
        
# Compute loss
loss = criterion(predictions, label)

# Convert predictions and labels to numpy for sklearn metrics
preds_np = predictions.detach().numpy()  # Detach predictions from the graph and convert to numpy
labels_np = label.numpy()  # Convert labels to numpy

# Get predicted classes
predicted_classes = np.argmax(preds_np, axis=1)

# Calculate Accuracy
acc = multi_accuracy(predictions, label.float())

# Calculate AUROC for each class (one-vs-rest approach)
auroc = roc_auc_score(labels_np, preds_np, multi_class="ovr")

# Ensure labels_np is in class label form, not one-hot encoded
if labels_np.ndim > 1:
    labels_np = np.argmax(labels_np, axis=1)


# Calculate Precision, Recall, and F1 Score (macro average for multi-class)
precision, recall, f1, _ = precision_recall_fscore_support(labels_np, predicted_classes, average='macro')

# Calculate AUPRC (Area Under the Precision-Recall Curve) for each class
auprc = average_precision_score(labels_np, preds_np, average="macro")

# Print results
print(f'\nAccuracy: {acc}\n')
# print(f'Loss: {loss}\n')
print(f'AUROC: {auroc}\n')
print(f'Precision (macro): {precision}\n')
print(f'Recall (macro): {recall}\n')
print(f'F1 Score (macro): {f1}\n')
print(f'AUPRC (macro): {auprc}\n')

tensor([1, 0, 7, 7, 0, 9, 9, 1, 0, 4, 1, 4, 4, 5, 0, 0, 6, 4, 5, 0, 9, 5, 4, 1,
        0, 1, 8, 1, 1, 0, 0, 9, 6, 5, 4, 1, 1, 8, 6, 4, 4, 0, 0, 0, 4, 1, 4, 4,
        4, 1, 5, 6, 0, 4, 0, 4, 4, 1, 0, 1, 1, 4, 0, 4, 4, 1, 0, 0, 4, 0, 0, 9,
        0, 4, 1, 0, 9, 0, 5, 1])
tensor([2., 5., 7., 7., 8., 5., 3., 4., 9., 6., 4., 2., 6., 5., 9., 2., 2., 8.,
        0., 9., 9., 0., 7., 7., 4., 5., 8., 1., 1., 7., 3., 9., 5., 3., 8., 5.,
        1., 8., 6., 9., 4., 0., 3., 3., 4., 4., 4., 9., 0., 1., 5., 6., 9., 5.,
        0., 1., 9., 1., 2., 7., 1., 8., 8., 4., 2., 6., 0., 0., 8., 0., 6., 2.,
        0., 4., 6., 3., 7., 7., 3., 1.], dtype=torch.float64)

Accuracy: 0.3499999940395355

AUROC: 0.7358316087545057

Precision (macro): 0.408930093776641

Recall (macro): 0.33551587301587305

F1 Score (macro): 0.2995844902224213

AUPRC (macro): 0.30781451086955824



/home/jaeyeob/anaconda3/envs/lstm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
